# Keymaker file
### Date: 09/12/19

This file is intended to address the issue of missing keys within the overall data. At the time of this writing, there remains a large selection of keys that do not find their way into the flat database. Goal is to gather up all keys, (regardless of how insignificant) and then later methodically drop them. The current program opts to set 1 organization as the primary form. While it allows for guarneteed consistency in one sense of the word, it rejects all other information that doesn't fit the primary form. So for instance, Monroe's HfH for the year 2018 has 470 keys. A database generated to fit this form would also have 470 keys. But suppose that in a prior year, HfH had 490 keys, or had keys that werent in the set of the 'original' 470 keys. Well, as the code stands, the program says "tough luck, can't give you that information!" This is an incredibly huge flaw. 

Additionally, there is an issue with how the code handles keys that are repeated. That is, when there is a list in the dictionary structure, the code can only grab the first item. For instance, the code reaches Part 8 line 2 and grabs only the firs row. On Monroe's 2018 990, that process would miss the Restore sales infomation. This problem isn't confinded to that one line either. It pops up numerous times. Ultimately, I will need to create a new retrieval approach to get the necessary data from these particular sections

## So how to fix the issue of getting all keys?

Simple. Just feed a bunch of different IRS 990 docs until most unique keys are found. Write some code that makes a unique list of these keys. But that solution immediately presents the problem of handling keys that are repeated (like the ones in lists)



In [24]:
import os, requests, re, time, numpy as np, pandas as pd, pickle, xmltodict
from IPython.display import clear_output
from random import sample

def get_node_names(parent):
    node_names = []
    
    for item in parent.items():
        if item[1] != None:
            if type(item[1]) == str:
                node_names.append(item[0])
            
            elif type(item[1]) == list:
                try:
                    child_list_node_names = get_node_names(item[1][0])
                    for child in child_list_node_names:
                        string_name = item[0] + '.' + child
                        node_names.append(string_name)
                except AttributeError:
                    pass
            
            else:
                child_node_names = get_node_names(item[1])
                for child in child_node_names:
                    string_name = item[0] + '.' + child
                    node_names.append(string_name)
            
    return node_names

def get_node_values(parent):
    value_list = []
    for item in parent.items():
        if item[1] != None:
            if type(item[1]) == str:
                value_list.append(item[1])
            
            elif type(item[1]) == list:
                try:
                    child_values = get_node_values(item[1][0])
                    for value in child_values:
                        value_list.append(value)
                except AttributeError:
                    print('encountered an error')
            
            else:
                child_values = get_node_values(item[1])
                for value in child_values:
                    value_list.append(value)
    
    return value_list

In [2]:
with open('affiliates annual links', 'rb') as infile:
    html_list = pickle.load(infile)

### Pick any orgs from the list

In [6]:
print(html_list[2056])
print(html_list[1345])

https://s3.amazonaws.com/irs-form-990/201621349349301677_public.xml
https://s3.amazonaws.com/irs-form-990/201530309349301113_public.xml


### 'Bricks' that I'll be sampling

In [33]:
sample_htmls = sample(html_list, 100)

In [34]:
sample_htmls

['https://s3.amazonaws.com/irs-form-990/201830259349301253_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201813179349306466_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201520249349300607_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201531059349300403_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201911349349304021_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201803179349304000_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201931299349301713_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201910819349300701_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201520489349301152_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201812299349300321_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201920159349301252_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201633019349300983_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201743459349300329_public.xml',
 'https://s3.amazonaws.com/irs-form-990/201531949349301313_publi

In [35]:
def to_brick(html):
    req = requests.get(html).content
    xml_dict_temp = xmltodict.parse(req)
    brick = dict(zip(get_node_names(xml_dict_temp),get_node_values(xml_dict_temp)))
    return brick


In [36]:
list_of_bricks = []
for html in sample_htmls:
    list_of_bricks.append(to_brick(html))

In [37]:
wall = pd.DataFrame(list_of_bricks)

In [38]:
wall

,Return.@returnVersion,Return.@xmlns,Return.@xmlns:xsi,Return.@xsi:schemaLocation,Return.ReturnData.@documentCnt,Return.ReturnData.@documentCount,Return.ReturnData.IRS990.@documentId,Return.ReturnData.IRS990.@referenceDocumentId,Return.ReturnData.IRS990.@referenceDocumentName,Return.ReturnData.IRS990.@softwareId,...,Return.ReturnHeader.ReturnTs,Return.ReturnHeader.ReturnType,Return.ReturnHeader.ReturnTypeCd,Return.ReturnHeader.TaxPeriodBeginDate,Return.ReturnHeader.TaxPeriodBeginDt,Return.ReturnHeader.TaxPeriodEndDate,Return.ReturnHeader.TaxPeriodEndDt,Return.ReturnHeader.TaxYear,Return.ReturnHeader.TaxYr,Return.ReturnHeader.Timestamp
0,2016v3.0,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,7,NaN,RetDoc1038000001,RetDoc1044400001,NaN,NaN,...,2018-01-25T15:24:28-06:00,NaN,990,NaN,2016-07-01,NaN,2017-06-30,NaN,2016,NaN
1,2017v2.2,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,6,NaN,IRS990,IRS990ScheduleO 990BinaryAttachments.1,NaN,17005038,...,2018-11-13T13:26:00-08:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN
2,2013v4.0,http://www.irs.gov/efile,NaN,NaN,NaN,6,IRS990,NaN,NaN,NaN,...,2015-01-24T11:08:38-05:00,NaN,990,NaN,2013-07-01,NaN,2014-06-30,NaN,2013,NaN
3,2013v4.0,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,NaN,NaN,5,RetDoc3,RetDoc6,NaN,NaN,...,2015-04-15T06:25:20-05:00,NaN,990,NaN,2013-07-01,NaN,2014-06-30,NaN,2013,NaN
4,2017v2.3,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,NaN,6,NaN,RetDoc3,RetDoc6,NaN,NaN,...,2019-05-14T11:27:30-05:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN
5,2017v2.3,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,8,NaN,RetDoc1038000001,RetDoc1044400001,NaN,NaN,...,2018-11-13T12:26:07-06:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN
6,2017v2.2,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,6,NaN,00000001,NaN,IRS990ScheduleO ReasonableCauseExplanation,17005306,...,2019-05-09T09:39:17-07:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN
7,2017v2.3,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,5,NaN,RetDoc1038000001,RetDoc1044400001,NaN,NaN,...,2019-03-22T07:39:02-05:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN
8,2013v4.0,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,NaN,NaN,4,RetDoc2,RetDoc5,NaN,NaN,...,2015-02-17T10:21:00-06:00,NaN,990,NaN,2013-07-01,NaN,2014-06-30,NaN,2013,NaN
9,2017v2.2,http://www.irs.gov/efile,http://www.w3.org/2001/XMLSchema-instance,http://www.irs.gov/efile,4,NaN,00000001,NaN,IRS990ScheduleO ReasonableCauseExplanation,17005306,...,2018-08-17T09:01:09-07:00,NaN,990,NaN,2017-07-01,NaN,2018-06-30,NaN,2017,NaN


In [39]:
for each in wall.keys():
    print(each)
    

Return.@returnVersion
Return.@xmlns
Return.@xmlns:xsi
Return.@xsi:schemaLocation
Return.ReturnData.@documentCnt
Return.ReturnData.@documentCount
Return.ReturnData.IRS990.@documentId
Return.ReturnData.IRS990.@referenceDocumentId
Return.ReturnData.IRS990.@referenceDocumentName
Return.ReturnData.IRS990.@softwareId
Return.ReturnData.IRS990.@softwareVersion
Return.ReturnData.IRS990.@softwareVersionNum
Return.ReturnData.IRS990.AccountantCompileOrReview
Return.ReturnData.IRS990.AccountantCompileOrReviewInd
Return.ReturnData.IRS990.AccountsPayableAccrExpnssGrp.BOYAmt
Return.ReturnData.IRS990.AccountsPayableAccrExpnssGrp.EOYAmt
Return.ReturnData.IRS990.AccountsPayableAccruedExpenses.BOY
Return.ReturnData.IRS990.AccountsPayableAccruedExpenses.EOY
Return.ReturnData.IRS990.AccountsReceivable.BOY
Return.ReturnData.IRS990.AccountsReceivable.EOY
Return.ReturnData.IRS990.AccountsReceivableGrp.BOYAmt
Return.ReturnData.IRS990.AccountsReceivableGrp.EOYAmt
Return.ReturnData.IRS990.AcctCompileOrReviewBasis

Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.CharitableContriOtherEventsAmt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.CharitableContributionsTotAmt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.DirectExpenseSummaryEventsAmt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.EntertainmentEvent1Amt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.EntertainmentTotalEventsAmt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.Event1Nm
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.Event2Nm
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.FoodAndBeverageEvent1Amt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.FoodAndBeverageEvent2Amt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.FoodAndBeverageOtherEventsAmt
Return.ReturnData.IRS990ScheduleG.FundraisingEventInformationGrp.FoodAndBeverageTotalEventsAmt
Return.ReturnDa